## Advanced ML: Домашнее задание 2

In [1]:
import pickle
import pandas as pd
import numpy as np

from datetime import datetime
from collections import defaultdict
from sklearn.linear_model import LogisticRegression, LinearRegression
from scipy.stats import kendalltau, spearmanr


from tqdm import tqdm

In [2]:
df_tournaments = pickle.load(open('/kaggle/input/data-for-made-ml-hw-2/tournaments.pkl', 'rb'))
df_results = pickle.load(open('/kaggle/input/data-for-made-ml-hw-2/results.pkl', 'rb'))
df_players = pickle.load(open('/kaggle/input/data-for-made-ml-hw-2/players.pkl', 'rb'))

1. Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
* взять в тренировочный набор турниры с dateStart из 2019 года; 
* в тестовый — турниры с dateStart из 2020 года.


In [11]:
train_df = []
test_df = []

for key, tournament in df_tournaments.items():
    not_defined_idxs = set()
    if tournament['dateStart'][:4] not in ['2019', '2020']:
        continue

    for team_result in df_results[key]:
        if 'mask' in team_result.keys() and team_result['mask']:
            for i, is_answered in enumerate(team_result['mask']):
                if is_answered in ['X', '?']:
                    not_defined_idxs.add(i)

    for team_result in df_results[key]:
        for player in team_result['teamMembers']:
            if 'mask' in team_result.keys() and team_result['mask']:
                for i, is_answered in enumerate(team_result['mask']):
                    if i not in not_defined_idxs:
                        data = (
        {
            'player_id': player['player']['id'],
            'player_name': f"{player['player']['surname']} {player['player']['name']} {player['player']['patronymic']}",
            'player_rating': player['rating'],
            'team_id': team_result['team']['id'],
            'team_position': team_result['position'],
            'tournament_id': key,
            'tournament_name': tournament['name'],
            'question_order': i,
            'is_answered': int(team_result['mask'][i])
        })
                        if tournament['dateStart'][:4] == '2019':
                            train_df.append(data)
                        elif tournament['dateStart'][:4] == '2020':
                            test_df.append(data)
                        else:
                            print(tournament['dateStart'][:4])

In [12]:
train_df = pd.DataFrame(train_df)
test_df = pd.DataFrame(test_df)

train_df.shape, test_df.shape

((20844798, 9), (4458857, 9))

In [13]:
# Топ 10 результативных игроков за 2020 год (из 2087 давших более 450 ответов)
df_top_ans = test_df.groupby('player_name').is_answered.agg(['count','sum'])
df_top_ans['effectiveness'] = df_top_ans['sum'] / df_top_ans['count']
df_top_ans[df_top_ans['count'] > 450].sort_values('effectiveness', ascending=False)[:10]

,count,sum,effectiveness
player_name,,,
Семушин Иван Николаевич,625,486,0.777600
Руссо Максим Михайлович,517,396,0.765957
Савченков Михаил Владимирович,670,509,0.759701
Тюнькин Игорь Вячеславович,1289,976,0.757176
Кафиатуллин Тимур Рустемович,724,547,0.755525
Крапиль Николай Валерьевич,518,390,0.752896
Поташев Максим Оскарович,636,475,0.746855
Вознесенский Сергей Георгиевич,671,501,0.746647
Сорожкин Артём Сергеевич,1668,1245,0.746403


2. Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
* повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
* в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
* для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


In [14]:
train_df['player_answer_rate'] = train_df.groupby('player_id')['is_answered'].transform('mean')

train_df['tournament_answer_rate'] = train_df.groupby('tournament_id')['is_answered'].transform('mean')

train_df['global_answer_rate'] = train_df['is_answered'].mean()

In [15]:
X_train = train_df[['player_answer_rate', 'global_answer_rate', 'tournament_answer_rate']]
y_train = train_df['is_answered']

In [16]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

3. Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
* предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
* в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

In [112]:
players_train = train_df.player_id.unique()
test_df_train = test_df[test_df.player_id.isin(players_train)]
test_teams = test_df_train[['player_id', 'team_id', 'tournament_id', 'team_position']].drop_duplicates()
test_teams['tournament_answer_rate'] = 1
test_teams.shape

(107091, 5)

In [113]:
player_answer_rete = train_df[['player_id', 'player_answer_rate', 'global_answer_rate']].drop_duplicates()
test_teams = test_teams.merge(player_answer_rete, how='left', on='player_id')

df_for_pred = test_teams[['player_answer_rate', 'global_answer_rate', 'tournament_answer_rate']]
test_teams['predict'] = model.predict_proba(df_for_pred)[:, 1]

In [142]:
test_teams['predict_inv'] = 1 - test_teams['predict']

ranks = test_teams.groupby(['team_id', 'tournament_id', 'team_position'])\
                  .agg({'predict_inv': 'prod'})\
                  .groupby(['team_id', 'tournament_id', 'team_position'])\
                  .agg({'predict_inv': 'sum'})\
                  .reset_index()

ranks['predict_inv'] = 1 - ranks['predict_inv']
ranks['rank'] = ranks.groupby('tournament_id')['predict_inv'].rank("dense", ascending=False)

tournament_ids = ranks['tournament_id'].unique()
kend_corrs = []
sp_corrs = []
for t in tournament_ids:
    rank_t = ranks[ranks['tournament_id'] == t]
    kend_corr = kendalltau(rank_t['rank'], rank_t['team_position'])[0]
    sp_corr = spearmanr(rank_t['rank'], rank_t['team_position'])[0]
    if not np.isnan(sp_corr):
        sp_corrs.append(sp_corr)
    if not np.isnan(kend_corr):
        kend_corrs.append(kend_corr)

print(f"Корреляция Спирмена: {np.mean(sp_corrs)}")
print(f"Корреляция Кендалла: {np.mean(kend_corrs)}")

Корреляция Спирмена: 0.6430556016505725
Корреляция Кендалла: 0.49121726540236477
